# 0. Setup Paths

In [2]:
# remove Tensorflow directory to reset
!rm -r Tensorflow

rm: cannot remove 'Tensorflow': No such file or directory


In [3]:
import os

In [4]:
# define constants to use
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [5]:
# define all the paths to use within the workspace
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'IMAGE_TRAIN_PATH': os.path.join('Tensorflow', 'workspace','images', 'train'),
    'IMAGE_VALIDATE_PATH': os.path.join('Tensorflow', 'workspace','images', 'valid'),
    'IMAGE_TEST_PATH': os.path.join('Tensorflow', 'workspace','images', 'test'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME),
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'),
    'INFERENCE_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'inference'),
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'),
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc'),
    'ROBOFLOW_TRAIN_PATH':os.path.join('Bird-Nest-15', 'train'),
    'ROBOFLOW_VALIDATE_PATH':os.path.join('Bird-Nest-15', 'valid'),
    'ROBOFLOW_TEST_PATH':os.path.join('Bird-Nest-15', 'test')
 }

In [6]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [7]:
# create the directories for all the paths defined
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# 1. Download TF Models Pretrained Models from Tensorflow Model Zoo and Install TFOD

In [8]:
# https://www.tensorflow.org/install/source_windows

In [9]:
# windows OS install wget
if os.name=='nt':
    !pip install wget
    import wget

In [10]:
# clone the Tensorflow object detection api repo
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 88134, done.
remote: Counting objects: 100% (1846/1846), done.
remote: Compressing objects: 100% (765/765), done.
remote: Total 88134 (delta 1211), reused 1643 (delta 1063), pack-reused 86288
Receiving objects: 100% (88134/88134), 603.02 MiB | 34.10 MiB/s, done.
Resolving deltas: 100% (63109/63109), done.


In [11]:
# Install Tensorflow Object Detection
if os.name=='posix':
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e .

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
Processing /content/Tensorflow/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# run the verification script
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [12]:
# for macOS users
!pip install tensorflow==2.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninstalling gast-0.5.4:
      Successfully uninstalled gast-0.5.4
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalled tensorboard-2.1

In [ ]:
!pip uninstall protobuf matplotlib -y
!pip install protobuf==3.20 matplotlib==3.2

In [13]:
import object_detection

In [ ]:
!pip list

In [14]:
# move pretrained model to workspace path
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2023-10-28 08:52:02--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.161.207, 74.125.126.207, 74.125.132.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.161.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20518283 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.57M  --.-KB/s    in 0.1s    

2023-10-28 08:52:02 (181 MB/s) - ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz’ saved [20518283/20518283]

ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_640x6

# 2. Create Label Map

In [15]:
# defining all the labels
labels = [{'name':'0', 'id':1, 'display_name': '0'}, {'name':'1', 'id':2, 'display_name': '1'}, {'name':'2', 'id':3, 'display_name': '2'}]

# open and write out a label map file
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('\tdisplay_name:\'{}\'\n'.format(label['display_name']))
        f.write('}\n')

# 3. Download images and TF records from Roboflow

In [16]:
# remove the bird nest image directory
!rm -r Bird-Nest-15

In [17]:
# download the dataset of ebn images from roboflow
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="1woWfE1q4RoyHytXmktz")
project = rf.workspace("ebn").project("bird-nest-exr6l")
dataset = project.version(15).download("tensorflow")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninstalled opencv-python-headless-4.8.1.78
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
      Successfully uninstalled idna-3.4
  Attempting uninstall: cycler
    Found existing installation: cycler

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Bird-Nest-15 in tensorflow:: 100%|██████████| 775/775 [00:00<00:00, 2424.58it/s]


In [18]:
# move the annotations CSV files to the annotation path
import glob

source_train = os.path.join(paths['ROBOFLOW_TRAIN_PATH'],'_annotations.csv')
destination_train = os.path.join(paths['ANNOTATION_PATH'], 'train.csv')

source_test = os.path.join(paths['ROBOFLOW_TEST_PATH'],'_annotations.csv')
destination_test = os.path.join(paths['ANNOTATION_PATH'], 'test.csv')

source_valid = os.path.join(paths['ROBOFLOW_VALIDATE_PATH'],'_annotations.csv')
destination_valid = os.path.join(paths['ANNOTATION_PATH'], 'valid.csv')

os.rename(source_train, destination_train)
os.rename(source_test, destination_test)
os.rename(source_valid, destination_valid)

In [19]:
# move all the images into the workspace train, validate, and test directories
import shutil

source_folders = [paths['ROBOFLOW_TRAIN_PATH'], paths['ROBOFLOW_VALIDATE_PATH'], paths['ROBOFLOW_TEST_PATH']]
destination_folders = [os.path.join(paths['IMAGE_PATH'], 'train'), os.path.join(paths['IMAGE_PATH'], 'valid'), os.path.join(paths['IMAGE_PATH'], 'test')]

for i in range(len(source_folders)):
  source_folder = source_folders[i]
  destination_folder = destination_folders[i]
  # fetch all files
  for file_name in os.listdir(source_folder):
      # construct full file path
      source = os.path.join(source_folder, file_name)
      destination = os.path.join(destination_folder, file_name)
      # move only files
      if os.path.isfile(source):
          shutil.move(source, destination)
          print('Moved:', file_name)

Moved: BrokenBig-3-_bmp.rf.c6f1d10047d887599e95078db2dcb4d3.jpg
Moved: SizeB-15-_bmp.rf.f928a9bfb340a2131702fd913b5babac.jpg
Moved: WhiteBeige-8-_bmp.rf.089dc74a553da57ee83a3020d2d5a71e.jpg
Moved: HeavyFeather-9-_bmp.rf.9139d52f57c3bcca284c4bb6c2943a4a.jpg
Moved: SizeB-8-_bmp.rf.d04481ae2b66897273c23e66fea17560.jpg
Moved: MedFeather-9-_bmp.rf.90e9f4bfcd0b6c4a0897c598bc6492b8.jpg
Moved: Yellow-19-_bmp.rf.87681c0976b7b05650fc2a83416f7e1d.jpg
Moved: Beige-11-_bmp.rf.d9de731a00c7e286cda3c70f5c0bc9e1.jpg
Moved: MedFeather-15-_bmp.rf.206629139c28657340e513a70a919b5e.jpg
Moved: Triangular-6-_bmp.rf.b0d23d9e2128bcd60fa2a3de37300516.jpg
Moved: White-7-_bmp.rf.9b65cac2f8d1c934766d1b4920023143.jpg
Moved: SizeA-4-_bmp.rf.6e0312a7d7a1aa6201ca61f977e6ff6b.jpg
Moved: WhiteBeige-13-_bmp.rf.89e591cdc0f9a0471f0f2c662a090fbe.jpg
Moved: LightFeather-8-_bmp.rf.a379d0b07f40ad4f127476f3d01583bc.jpg
Moved: BrokenSmall-34-_bmp.rf.07ad6ae9406d57150a81ad2347082c27.jpg
Moved: SizeA-13-_bmp.rf.6fa66596a4d7f6f65620

In [20]:
# remove the bird nest image directory
!rm -r Bird-Nest-15

In [21]:
# download the tfrecord files from roboflow
rf = Roboflow(api_key="1woWfE1q4RoyHytXmktz")
project = rf.workspace("ebn").project("bird-nest-exr6l")
dataset = project.version(15).download("tfrecord")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Bird-Nest-15 in tfrecord:: 100%|██████████| 11/11 [00:00<00:00, 18.26it/s]


In [23]:
# move the tfrecord files to the annotation path
source_train = os.path.join(paths['ROBOFLOW_TRAIN_PATH'],'Bird-Nest.tfrecord')
destination_train = os.path.join(paths['ANNOTATION_PATH'], 'train.record')

source_test = os.path.join(paths['ROBOFLOW_TEST_PATH'],'Bird-Nest.tfrecord')
destination_test = os.path.join(paths['ANNOTATION_PATH'], 'test.record')

source_valid = os.path.join(paths['ROBOFLOW_VALIDATE_PATH'],'Bird-Nest.tfrecord')
destination_valid = os.path.join(paths['ANNOTATION_PATH'], 'valid.record')

os.rename(source_train, destination_train)
os.rename(source_test, destination_test)
os.rename(source_valid, destination_valid)

In [24]:
# remove the bird nest image directory
!rm -r Bird-Nest-15

# 4. Copy Model Config to Training Folder

In [25]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

# 5. Update Config For Transfer Learning

In [26]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [27]:
# get current configurations
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
config

In [28]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [29]:
# updating the config file
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [30]:
# write out update to the config file
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)

# 6. Train the model

In [31]:
# define the training script path
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [32]:
# define the train function
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=5000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [33]:
# train the model
!{command}

2023-10-28 08:54:03.908834: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-28 08:54:08.947954: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I1028 08:54:08.949138 139013598367744 mirrored_strategy.py:419] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 5000
I1028 08:54:08.973775 139013598367744 config_util.py:552] Maybe overwriting train_steps: 5000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1028 08:54:08.973968 139013598367744 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W1028 08:54:09.002936 139013598367744 dep

# 7. Evaluate the Model

In [34]:
# define the evaluate function
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [ ]:
# evaluate the model
!{command}

2023-10-28 09:06:05.713285: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
W1028 09:06:09.178299 132085040590848 model_lib_v2.py:1089] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I1028 09:06:09.178540 132085040590848 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1028 09:06:09.178631 132085040590848 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1028 09:06:09.178720 132085040590848 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W1028 09:06:09.178837 132085040590848 model_lib_v2.py:1106] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
2023-10-28 09:06:10.190238: W tensorflow/core/common_runtime/gpu/gpu_bfc_alloca

# 8. Load Train Model From Checkpoint

In [ ]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.builders import model_builder
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 9. Detect from an Image

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# create category index from label map for visualisation
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
# define the path of the image to be passed into model
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'train', 'BrokenBig-10-_bmp.rf.889d5780aa1c7e6ed1c837a23f7bd76a.jpg')

In [ ]:
# install arial for label purposes
!wget https://freefontsdownload.net/download/160187/arial.zip
!unzip arial.zip -d .

In [ ]:
from object_detection.utils import visualization_utils as viz_utils

# read image and convert to tensor
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)
input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

# detect within the image
detections = detect_fn(input_tensor)

# get the number of detections and the detections array
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

# define label offset
label_id_offset = 1
image_np_with_detections = image_np.copy()

# draw bounding box on the image passed in based on the configurations
viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            agnostic_mode=False,
            line_thickness=8,
            min_score_thresh=0.4)

# show the image
plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

# print out the detected class and confidence level
print(f"Detected class: {detections['detection_classes'][0]}\nConfidence level: {detections['detection_scores'][0] * 100}")

# Real Time Detections from your Webcam

In [ ]:
!pip uninstall opencv-python-headless -y

In [ ]:
# define the dimensions
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened():
    # read frame from webcam and convert to tensor
    ret, frame = cap.read()
    image_np = np.array(frame)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

    # detect within the frame
    detections = detect_fn(input_tensor)

    # get the number of detections and the detections array
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    # define label offset
    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    # draw bounding box on the image passed in based on the configurations
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                agnostic_mode=False)

    # show the result live
    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))

    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

## Generate Confusion Matrix

In [ ]:
types = ('*.png', '*.jpg') # the tuple of file types
files_grabbed = []
for file in types:
 files_grabbed.extend(glob.glob(os.path.join(paths['IMAGE_PATH'], 'test', file)))

In [ ]:
# define test images paths constant
TEST_IMAGE_PATHS = files_grabbed

In [ ]:
# labels array
labels = ["0", "1", "2"]

In [ ]:
import pandas as pd
test_pred = pd.DataFrame()

In [ ]:
from PIL import Image
import warnings
warnings.filterwarnings('ignore') # Suppress Matplotlib warnings

def load_image_into_numpy_array(image_path):
  return np.array(Image.open(image_path))

In [ ]:
import numpy as np

# loop through all the test images
for image_path in TEST_IMAGE_PATHS:

  # load the image using numpy
  image_np = load_image_into_numpy_array(image_path)

  # rename test image path and get the height and width of the image
  image_path = image_path.replace("Tensorflow/workspace/images/test/", "")
  height, width, color = image_np.shape

  # convert images to tensor
  input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

  # detect within the image
  detections = detect_fn(input_tensor)

  # get the number of detections and the detections array
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
  detections['num_detections'] = num_detections
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  # create a new row of detection information to be written out to a CSV file
  ll = []
  ll.append(image_path)
  ll.append(width)
  ll.append(height)
  ll.append(detections['detection_boxes'][0].tolist())
  ll.append(labels[int(detections['detection_classes'][0])])
  ll.append(detections['detection_scores'][0])

  pre_df = pd.DataFrame([{'filename':ll[0],
    'width':ll[1],
    'height':ll[2],
    'class':ll[4],
    'xmin':ll[3][1] * width,
    'ymin':ll[3][0] * height,
    'xmax':ll[3][3] * width,
    'ymax':ll[3][2] * height,
    'score':ll[5]}])

  # append to the test prediction array
  test_pred = pd.concat([test_pred, pre_df], axis=0)

In [ ]:
# write out prediction results to CSV file for confusion matrix generation
test_pred.to_csv(os.path.join(paths['ANNOTATION_PATH'], 'test_pred.csv'), index=False)

In [ ]:
"""
Created on Mon Sep 14 21:42:19 2020
@author: sujith
"""
import pandas as pd
import numpy as np

# define paths
test = pd.read_csv(os.path.join(paths['ANNOTATION_PATH'], 'test.csv'))
pred = pd.read_csv(os.path.join(paths['ANNOTATION_PATH'], 'test_pred.csv'))
test['filename'] = test['filename'].apply(lambda x : x.split('/')[-1])
pred['filename'] = pred['filename'].apply(lambda x : x.split('/')[-1])
output_path = os.path.join(paths['ANNOTATION_PATH'], 'confusion_matrix.csv')
categories = [0, 1, 2]

def process_detections(test, categories):
    confusion_matrix = np.zeros(shape=(len(categories) + 1, len(categories) + 1))
    file_unique = test['filename'].unique()
    for file in file_unique:
        # get the actual row of detection results of the test image
        test_df = test[test['filename']==file]
        test_df.reset_index(inplace = True, drop = True)

        # get the predicted row of detection results of the test image
        pred_df = pred[pred['filename']==file]
        pred_df.reset_index(inplace = True, drop = True)

        # get the actual class of the test image
        test_class = test_df['class']
        # get the predicted class of the test image
        pred_class = pred_df['class']

        # increment in confusion matrix
        confusion_matrix[int(pred_class)][int(test_class)] += 1

    return confusion_matrix

"""
Display the confusion matrix output and other important metrics (precision and recall) by class.
"""
def display(confusion_matrix, test, categories, output_path):
    results = []
    class_uniq = test['class'].unique()
    for label in class_uniq:
        class_id = int(float(categories[label]))
        name = label

        total_target = np.sum(confusion_matrix[class_id,:])
        total_predicted = np.sum(confusion_matrix[:,class_id])

        precision = float(confusion_matrix[class_id, class_id] / total_predicted)
        recall = float(confusion_matrix[class_id, class_id] / total_target)

        results.append({'category' : name, 'precision': precision, 'recall' : recall})
    print(confusion_matrix)
    print(precision)
    print(recall)
    df = pd.DataFrame(results)
    print(df)
    df.to_csv(output_path)

if __name__ == '__main__':
    confusion_matrix = process_detections(test,categories)
    display(confusion_matrix, test, categories, output_path)

# 10. Freezing the Graph

In [ ]:
# define the freeze script path
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

In [ ]:
# define freeze function for model weights
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT , files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])

In [ ]:
# run the freeze function
!{command}

In [ ]:
# zip the savedmodel file
command = "zip -r {} {}".format(os.path.join(paths['OUTPUT_PATH'], 'saved_model.zip'), os.path.join(paths['OUTPUT_PATH'], 'saved_model'))
!{command}

# 11. Conversion to TFJS

In [ ]:
!pip install tensorflowjs

In [ ]:
# define function to export model to tfjs format
command = "tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default {} {}".format(os.path.join(paths['OUTPUT_PATH'], 'saved_model'), paths['TFJS_PATH'])

In [ ]:
# export model to tfjs format
!{command}

# 12. Conversion to TFLite

In [ ]:
# define path to script for exporting to frozen tflite 
TFLITE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'export_tflite_graph_tf2.py ')

In [ ]:
# define function to export model to frozen tflite format
command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(TFLITE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['TFLITE_PATH'])

In [ ]:
# export model to frozen tflite format
!{command}

In [ ]:
# define paths
FROZEN_TFLITE_PATH = os.path.join(paths['TFLITE_PATH'], 'saved_model')
TFLITE_MODEL = os.path.join(paths['TFLITE_PATH'], 'saved_model', 'detect.tflite')

In [ ]:
# define function to convert frozen tflite file to tflite file
command = "tflite_convert \
--saved_model_dir={} \
--output_file={} \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops".format(FROZEN_TFLITE_PATH, TFLITE_MODEL, )

In [ ]:
# convert frozen tflite file to tflite file
!{command}

# 13. Zip and Export Models

In [ ]:
# zip model
!tar -czf models.tar.gz {paths['CHECKPOINT_PATH']}

In [ ]:
# mount on google drive
from google.colab import drive
drive.mount('/content/drive')